# Dependency Parsing
Build a neural transition-based dependency parser, based on the paper <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>.

We recommend runing this notebook on Google Colab instead of your local computer to avoid the hassle of installing necessary Python packages on local machine, and to get free GPU. Selecting "GPU" as the runtime type as this will speed up the training of your models. You can find this by going to <TT>Runtime > Change Runtime Type</TT> and select "GPU" from the dropdown menu.


# Step 1: Prepare Data




In [ ]:
import numpy as np
import random
import cloudpickle as cp
from urllib.request import urlopen

## Load Data

Here are some links about the data:
* We use one of the Universal Dependencies datasets from http://universaldependencies.org/docsv1/. Specifically, we use the UD_English dataset in version 1.4.
* Refer to https://universaldependencies.org/ if you want to know more about the Universal Dependencies framework in general.
* The data license can be found here: https://lindat.mff.cuni.cz/repository/xmlui/page/licence-UD-1.4

Run the following cells to load the data and see the number of sentences.

In [ ]:
def load_data():
    # Note: these are the URLs from my own google drive, but tested it is assesible from any other google drive account  
    train_set = cp.load(urlopen("https://drive.google.com/uc?export=download&id=14RF97hFa42JqGn2K3Ew-RAr-YIgCQO6P"))
    test_set = cp.load(urlopen("https://drive.google.com/uc?export=download&id=1WsYkm25JmqnFOJWk7KANPQ2TXNEBl8jE"))

    return train_set, test_set

In [ ]:
if __name__ == '__main__':
    train_set, test_set = load_data()
    print("Num. Train Examples:", len(train_set))
    print("Num. Test Examples: ", len(test_set))

Num. Train Examples: 12543
Num. Test Examples:  2002


## Build Vocabulary

Next, we build the `Vocabulary` class. This maps each word, part-of-speech tag (POS), and label to an id (index), which we will later use in the embeddings. We also need to enumerate each possible transition, and map it to an id, since this is what the neural network will try to predict. The `Vocabulary` class does this as follows:

Suppose there are $n$ labels. For each label, a Left-Arc (LA) and Right-Arc (RA) action are created for that label; and a separate Shift (S) action ids also created. This creates a total of $2n+1$ actions that the dependency parser will be able to choose from.

In [ ]:
class Vocabulary(object):
    def __init__(self, dataset):

        UNK = '<UNK>'
        NULL = '<NULL>'
        ROOT = '<ROOT>'

        # Find the label of the root
        root_labels = list([l for ex in dataset for (h, l) in zip(ex['head'], ex['label']) if h == 0])
        assert len(set(root_labels)) == 1
        self.root_label = root_labels[0]

        # Create mapping from transitions to ids
        labels = sorted(list(set([w for ex in dataset for w in ex['label'] if w != self.root_label]))) # list of unique non-root labels
        labels = [self.root_label] + labels # add root label too
        self.n_labels = len(labels)

        transitions = ['LA-' + l for l in labels] + ['RA-' + l for l in labels] + ['S']
        self.n_trans = len(transitions) # 2*n_labels + 1
        self.tran2id = {t: i for (i, t) in enumerate(transitions)}
        self.id2tran = {i: t for (i, t) in enumerate(transitions)}

        # Create mapping from word, pos, & label to id
        # Do labels first
        self.LABEL_PREFIX = '<l>:'
        self.tok2id = {self.LABEL_PREFIX + l: i for (i, l) in enumerate(labels)}
        self.LABEL_NULL = self.tok2id[self.LABEL_PREFIX + NULL] = len(self.tok2id) # Add null label in

        # Do POS's
        self.POS_PREFIX = '<p>:'
        all_pos = sorted(set([self.POS_PREFIX + w for ex in dataset for w in ex['pos']])) # Get pos's
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_pos)}) # Add poses in
        self.POS_NULL = self.tok2id[self.POS_PREFIX + NULL] = len(self.tok2id) # Add null pos
        self.POS_ROOT = self.tok2id[self.POS_PREFIX + ROOT] = len(self.tok2id) # Add root pos
        self.n_pos = 2 + len(all_pos) # +3 for null, root

        # Do words
        all_word = sorted(set([w for ex in dataset for w in ex['word']]))
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_word)}) # Add words in
        self.WORD_UNK = self.tok2id[UNK] = len(self.tok2id) # Add unk word
        self.WORD_NULL = self.tok2id[NULL] = len(self.tok2id) # Add null word
        self.WORD_ROOT = self.tok2id[ROOT] = len(self.tok2id) # Add root word
        self.n_words = 3 + len(all_word) # +3 for unk, null, root

        self.id2tok = {v: k for (k, v) in self.tok2id.items()} # Flip it
        self.n_tokens = len(self.tok2id)

    def printStats(self):
        print('Num. labels:', self.n_labels)
        print('Num. transitions (2*n_labels + 1):', self.n_trans)
        print('Num. pos:', self.n_pos)
        print('Num. words:', self.n_words)
        print('Num. tokens:', self.n_tokens)


    def buildSentences(self, examples):
        processed_sentences = []
        for ex in examples:
            # Initialize words & dependencies
            words = [Word('<ROOT>', '<POS_ROOT>', 0, self.WORD_ROOT, self.POS_ROOT)]
            deps = [] 
          
            # Loop over words in sentence
            for i  in  range(len(ex['word'])):
                w = ex['word'][i]
                word_id = (self.tok2id[w] if w in self.tok2id else self.WORD_UNK)
                pos = self.POS_PREFIX + ex['pos'][i]
                pos_id = self.tok2id[pos]
                word = Word(ex['word'][i], ex['pos'][i],i+1, word_id, pos_id)

                words.append(word)
                deps.append((ex['head'][i], word, ex['label'][i] ))
            
            # Create dependencies
            dependencies = Dependencies(self)
            for dep in deps:
                dependencies.add(words[dep[0]], dep[1], dep[2])

            processed_sentences.append((words, dependencies))
                      
        return processed_sentences

Run the following cell to see some stats from the vocabulary.

In [ ]:
if __name__ == '__main__':
    Vocabulary(train_set).printStats()

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735


# Step 2: Parser Data Structures
In this section, some useful data structures for dependency parsing are defined. In particular, the `Stack` and `Buffer` structures are defined, which make up a `ParserConfiguration`. A function to update these data structures based on a particular transition is implemented also.

## Data Structures

Some useful data classes are defined. 

In [ ]:
class Word(object):
    '''
    Represents a word in the sentence.
    '''

    def __init__(self, word, pos, idx, word_id=None, pos_id=None):
        self.word = word
        self.pos = pos
        self.idx = idx
        self.word_id = word_id
        self.pos_id = pos_id

    def __str__(self):
        return 'Word(idx=' + str(self.idx) + ", word='" + self.word+"', pos='"+self.pos+"', word_id="+str(self.word_id)+', pos_id='+ str(self.pos_id) +')'

    def copy(self):
        return Word(self.word, self.pos, self.idx, self.word_id, self.pos_id)

    def __eq__(self, obj):
        if not isinstance(obj, Word): return False
        if obj.idx == self.idx:
            assert obj.word == self.word and obj.pos == self.pos and obj.word_id == self.word_id and obj.pos_id == self.pos_id, 'Your Word object has been corrupted.'
        return obj.idx == self.idx


class Arc(object):
    '''
    Represents an arc between two words.
    '''

    def __init__(self, head, dependent, label, label_id):
        self.head=head # Word object
        self.dependent=dependent # Word object
        self.label=label
        self.label_id = label_id

    def __str__(self):
        return 'Arc(head_idx='+str(self.head.idx)+', dep_idx='+str(self.dependent.idx)+', label_id='+ str(self.label_id)+')'


class Dependencies(object):
    '''
    Represents the dependency arcs in a sentence.
    '''

    def __init__(self, vocab):
        self.arcs = []
        self.vocab = vocab
        self.dep_to_head_mapping = {} # For fast lookup
    
    def add(self, head, dependent, label):
        '''
        Add a dependency from head to dependent with label.
        Inputs:
            head: Word object
            dependent: Word object
            label: str
        '''
        assert label[:3] != 'LA-' and label[:3] != 'RA-', 'You need to pass in just the label to add(...), not the entire action.'
        assert head is not None and dependent is not None, "You must pass in two Word objects to add(...)."
        self.arcs.append(Arc(head, dependent, label, self.vocab.tok2id[self.vocab.LABEL_PREFIX+label]))
        assert dependent.idx not in self.dep_to_head_mapping
        self.dep_to_head_mapping[dependent.idx] = self.arcs[-1]

    def getArcToHead(self, dependent):
        '''
        Returns the Arc that connects the head of dependent to dependent.
        Inputs:
            dependent: Word object
        '''
        if dependent.idx == 0: # Special case for ROOT
            return Arc(None, dependent, None, None)
        return self.dep_to_head_mapping[dependent.idx]

    def __iter__(self):
        return iter(self.arcs) # Allows to iterate "for x in Dependencies(...)"

## Stack & Buffer

Define stack and buffer data structures, implement the `push(...)` and `pop(...)` methods of the `Stack`, and the `pop(...)` method of the `Buffer`. 

In [ ]:
class Stack(object):
    def __init__(self, input=[]):
        '''
        Initialize an (empty) stack.
        '''
        self.stack = [word.copy() for word in input] 

    def push(self, item):
        '''
        Push item onto (the end of) self.stack. Returns nothing.
        '''

        self.stack.append(item)

    def pop(self):        
        '''
        Pop item from (the end of) self.stack. Returns the item popped.
        '''
        assert len(self.stack) > 0

        # remove the last item from the list self.stack, and return that item
        item = self.stack.pop(-1)
        return item

    def get_si(self, i):
        '''
        Returns si (the ith element of the stack) if it exists, otherwise None.
        '''
        assert i > 0, 'Must provide i > 0'
        return self.stack[-i] if len(self.stack) >= i else None

    def __getitem__(self, idx):
        return self.stack[idx]

    def __len__(self):
        return len(self.stack)

    def __str__(self):
        return str([str(x) for x in self.stack])


class Buffer(object):
    def __init__(self, sentence):
        '''
        Initialize as a list of words in sentence.
        '''
        self.buffer = [word.copy() for word in sentence]

    def pop(self):
        '''
        Pop item from (the beginning of) self.buffer. Returns the item popped.
        '''
        assert len(self.buffer) > 0
        
        item = self.buffer.pop(0)
        return item

    def get_bi(self, i):
        '''
        Returns bi (the ith element of the buffer) if it exists, otherwise None.
        '''
        assert i > 0, 'Must provide i > 0'
        return self.buffer[i-1] if len(self.buffer) >= i else None

    def __getitem__(self, idx):
        return self.buffer[idx]

    def __len__(self):
        return len(self.buffer)

    def __str__(self):
        return str([str(x) for x in self.buffer])

## Parser Configuration

Next, a `ParserConfiguration` class is created, which contains the `Stack`, `Buffer`, and `Dependencies` data structures. 

More specifically, let $\sigma$ represent the stack, $\beta$ the buffer, and $A$ the set of arcs (dependencies). Based on the value of `transition`, `parse_step(self, transition)` should do the following:
* `transition = 'S'`: &nbsp;<b>Shift</b> $w_k$ from the buffer to the stack. $(\sigma, w_k|\beta, A) \Rightarrow (\sigma|w_k, \beta, A)$
* `transition = 'LA-label'`: &nbsp;Add a <b>left arc</b> with label $label$ from $w_j$ to $w_i$. $(\sigma |w_i w_j , \beta, A) \Rightarrow (\sigma |w_j, \beta, A \cup \{(w_j, label, w_i)\})$
* `transition = 'RA-label'`: &nbsp;Add a <b>right arc</b> with label $label$ from $w_i$ to $w_j$. $(\sigma |w_i w_j , \beta, A) \Rightarrow (\sigma |w_i, \beta, A \cup \{(w_i, label, w_j)\})$


In [ ]:
class ParserConfiguration(object):
    def __init__(self, sentence, vocab):
        '''
        Inputs:
            sentence: list of Word objects
            vocab: Vocabulary object
        '''

        self.vocab = vocab

        assert sentence[0].word_id == self.vocab.WORD_ROOT
        # Initialize stack with ROOT
        self.stack = Stack([sentence[0]]) 
        # Initialize buffer with sentence
        self.buffer = Buffer(sentence[1:]) 
        self.dependencies = Dependencies(vocab)

    def parse_step(self, transition):
        '''
        Update stack, buffer, and dependencies based on transition.
        Inputs:
            transition: str, "S", "LA-label", or "RA-label", where label is a valid label
        '''
        assert transition in self.vocab.tran2id


        if transition == "S": 
            # shift
            word = self.buffer.pop()
            self.stack.push(word)
        elif transition.startswith("LA-"):
            # Add a left arc - i.e arc from the top word to the 2nd word on stack, remove the 2nd word from stack
            # pop twice from the stack to remove the top two words, and then push back the top word
            # that is equivalent to removing the second word from the stack, then add the arc with the label           
            # transition[3:] is the label - starting index 3 to skip the first 3 chars "LA-" in transtion 
            top_word = self.stack.pop()    # pop off the top word from stack
            second_word = self.stack.pop() # pop off the 2nd word
            self.stack.push(top_word)
            self.dependencies.add(top_word, second_word, transition[3:])
        elif transition.startswith("RA-"):
            # Remove the top word from stack, add a right arc -  i.e. arc from the 2nd word to the top word on stack
            top_word = self.stack.pop()    # pop off the top word from stack
            # the second word of the original stack (before removing the top word) is now the top word - get_si(1)
            second_word = self.stack.get_si(1)
            self.dependencies.add(second_word, top_word, transition[3:]) 
        else:
            # Something wrong with the transtion, print a warning message
            print("transtion error - " + transition)


# Step 3: Generate Training Data

As shown above, the dataset contains many sentences along with their gold dependency parses. In order to use a transition-based parsing algorithm, the next parser action at each time step must be predicted, based on the current parser configuration. Thus, each sentence needs to be converted into a series of training examples, where the input features are based on the current parser configuration and the correct label is the gold action.

In this section, we first implement an algorithm to select the next parser action at each time step based on the gold dependency parse. Then, we will extract features from each step's parser configuration, which the neural network will use to predict the next action.

## Compute Gold Action

Next, we will write a function `get_gold_action(stack, buffer, gold_dependencies)`. Given a stack and buffer, this function should return the next action of the parser based on the gold dependencies. 

Let $s_i$ be $i$th element of the stack and $h(s_i)$ be the head word of $s_i$. The pseudocode is as follows:

1. If the stack only contains `ROOT`:
 - If the buffer is not empty, return `S` (shift).
 - If the buffer is empty, return `DONE`, indicating the parse is complete.
2. If $h(s_2)=s_1$, return `LA-label`. Here, `label` is the label of the arc that attaches $s_2$ to $s_1$.
3. If $h(s_1)=s_2$ <b>and</b> $h(b_i) \neq s_1$ for all words $b_i$ in the buffer, return `RA-label`. Here, `label` is the label of the arc that attaches $s_1$ to $s_2$.
 - This condition means that you cannot attach $s_1$ until everything in the buffer that depends on $s_1$ is attached. 
4. Otherwise:
 - If the buffer is not empty, return `S`. 
 - If the buffer is empty, return `None`, indicating a failed parse (i.e., the sentence is non-projective).


In [ ]:
def get_gold_action(stack, buffer, gold_dependencies):
    '''
    Given stack & buffer, compute the next gold action based on gold_dependencies.
    Args:
        - stack: Stack object
        - buffer: Buffer object
        - gold_dependencies: Dependencies object
    Returns:
        - action: str; 'S', 'LA-label', or 'RA-label', where 'label' is a valid label. Return None if no action possible and 'DONE' if the parse is complete.
    '''


    # 1. if stack only cntains ROOT, i.e. len(stack) == 1
    if len(stack) == 1: #len() can be called because __len__ is defined in Stack class
        if len(buffer) > 0:     
            return 'S'
        else:
            return 'DONE'
    
    # 2. If h(s2) = s1 return LA-label
    s1 = stack.get_si(1)
    s2 = stack.get_si(2)
    arc = gold_dependencies.getArcToHead(s2)
    if arc.head == s1:
        action = "LA-" + arc.label
        return action

    # 3. If h(s1) = s2 and h(bi) /= s1 for all ... return RA-label
    arc = gold_dependencies.getArcToHead(s1)
    if arc.head == s2:
        count = 0
        for bi in buffer:    # this for loop can be used because __getitem__ is defined in Buffer class
            if gold_dependencies.getArcToHead(bi).head == s1:
                count += 1
                break
        if count == 0: 
        # There exist no bi such that h(bi) == s1, i.e. h(bi) /= s1 for all bi in buffer
            action = "RA-" + arc.label
            return action
    # 4. Otherwise:
    if len(buffer) > 0:
        return 'S'
    else:
        return None


## Generate Training Examples


Now implement a function to generate the training examples. Each sentence needs to be converted into a series of separate training examples. Each training example will essentially be a partial parser configuration along with its gold action; the goal of the neural network will be to predict this action from the parser configuration.

In order to make this prediction, we need to extract features from the parser configuration. We will implement the feature extraction method in a future section; for now, we pass in a dummy function `feat_extract(parser_config)` that returns an empty feature list.


In [ ]:
def generate_training_examples(sentence, gold_dependencies, vocab, feat_extract = lambda parser_config: []):
    '''
    Create training instances for sentence.
    Inputs:
        sentence: list of Word objects
        gold_dependencies: Dependencies object that contains the complete gold dependency tree
        vocab: Vocabulary object
        feat_extract: Feature extraction function
    Outputs:
        training_examples: List of tuples (features, label), where features is a list and label is a string
    '''

    training_examples = []

    # (1) Initialize parser configuration (note that the __init__ method of ParserConfiguration creates the stack & buffer)
    parser_config = ParserConfiguration(sentence, vocab)
    # (2) Repeatedly call get_gold_action(...) on the current parser confuration until the gold action is 'DONE'
    gold_action = get_gold_action(parser_config.stack, parser_config.buffer, gold_dependencies)
    while gold_action != 'DONE':
          # (3) If the gold action is None at any step, return []  
          # (indicating the sentence cannot be parsed; it is non-projective)
          if gold_action == None:
              training_examples = []
              break
          # (4) Otherwise, append tuple (features, gold action) to training_examples, 
          # where features is the result of calling feat_extract on the parser configuration
          features = feat_extract(parser_config)
          training_examples.append((features, gold_action))
          #(5) Update the parser configuration according to the gold action
          parser_config.parse_step(gold_action) 
          gold_action = get_gold_action(parser_config.stack, parser_config.buffer, gold_dependencies)
    # (6) Return training_examples
    return training_examples

The following function calls `generate_training_examples(...)` on every sentence in the dataset to create the full training data.

In [ ]:
def generate_all_training_examples(vocab, sentences, feat_extract = lambda parser_config: []):
    '''
    Generate training examples for all sentences.
    '''
    all_training_examples = []
    successful_sents = 0
    for sentence in sentences:
        training_examples = generate_training_examples(sentence[0], sentence[1], vocab, feat_extract)
        if training_examples != []:
            all_training_examples += training_examples
            successful_sents += 1

    print("Successfully generated training examples for", successful_sents, "/", len(sentences), "sentences")
    print("Number of training examples:", len(all_training_examples))

    return all_training_examples

In [ ]:
if __name__ == '__main__':
    _vocab = Vocabulary(train_set)
    generate_all_training_examples(_vocab, _vocab.buildSentences(train_set))    


Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


## Extract Features
By this point, we have written code to create individual training instances. Each instance is made up of a parser configuration along with the gold action that the classifier should be trained to predict.

In order to make this prediction, the neural network will have to rely on features extracted from each parser configuration. We follow the procedure described at the end of Section 3.1 of <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>. In total, we will extract 48 features from the parser configuration $-$ 18 word features, 18 POS features, and 12 label features:
* Word & POS features for $s_1$, $s_2$, $s_3$ (top 3 items of the stack)
* Word & POS features for $b_1$, $b_2$, $b_3$ (top 3 items of the buffer)
* Word, POS, & label features for $lc_1(s_1)$, $lc_2(s_1)$, $lc_1(s_2)$, $lc_2(s_2)$ (the first & second leftmost children of the top 2 items on the stack)
* Word, POS, & label features for $rc_1(s_1)$, $rc_2(s_1)$, $rc_1(s_2)$, $rc_2(s_2)$ (the first & second rightmost children of the top 2 items on the stack)
* Word, POS, & label features for $lc_1(lc_1(s_1))$, $lc_1(lc_1(s_2))$, $rc_1(rc_1(s_1))$, $rc_1(rc_1(s_2))$ (the leftmost of the leftmost & rightmost of the rightmost children of the top 2 items on the stack)

We will write a separate function for each of the 5 bullets above. Each function will return a list of word features, a list of POS features, and (in the relevant cases) a list of label features. 

A "feature" refers to the id (index in the Vocabulary) of a word, POS, or label. The neural network will then construct embeddings for each id.

First, write 2 functions to extract features corresponding to the words at the top of the stack & buffer:
* `get_top3_stack_features(parser_config)`: Return word & POS features (ids) for $s_1$, $s_2$, $s_3$ (top 3 words on the stack).
* `get_top3_buffer_features(parser_config)`: Return word & POS features (ids) for $b_1$, $b_2$, $b_3$ (top 3 words on the buffer).

Wherever a particular word does not exist (such as when the stack or buffer has length $< 3$) use the appropriate NULL token. This is necessary to ensure that the neural network will see an equally sized feature vector for each example.

In [ ]:
def get_top3_stack_features(parser_config):
    '''
    Get the word and POS features for s1, s2, and s3 (the top 3 items on the stack)
    Returns:
        word_features: List of word ids for s1, s2, s3 (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for s1, s2, s3 (use vocab.POS_NULL if a word does not exist)
    '''

    stack = parser_config.stack
    vocab = parser_config.vocab
    if len(stack) >= 3:
        s1 = stack.get_si(1)
        s2 = stack.get_si(2)
        s3 = stack.get_si(3)
        word_features = [s1.word_id, s2.word_id, s3.word_id]
        pos_features  = [s1.pos_id, s2.pos_id, s3.pos_id]

    elif len(stack) == 2:
        s1 = stack.get_si(1)
        s2 = stack.get_si(2)
        word_features = [s1.word_id, s2.word_id, vocab.WORD_NULL]
        pos_features  = [s1.pos_id, s2.pos_id, vocab.POS_NULL]

    elif len(stack) == 1:
        s1 = stack.get_si(1)
        word_features = [s1.word_id, vocab.WORD_NULL, vocab.WORD_NULL]
        pos_features  = [s1.pos_id, vocab.POS_NULL, vocab.POS_NULL]
    else:   
        # i.e. len(stack) == 0, this should not happen, since stack should at least have ROOT
        # If it happens, print warning message
        print("Inside get_top3_stack_features, len(stack) == 0")

    return word_features, pos_features

In [ ]:
def get_top3_buffer_features(parser_config):
    '''
    Get the word and POS features for b1, b2, and b3 (the top 3 items on the buffer)
    Returns:
        word_features: List of word ids for b1, b2, b3 (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for b1, b2, b3 (use vocab.POS_NULL if a word does not exist)
    '''

    buffer = parser_config.buffer
    vocab = parser_config.vocab
    if len(buffer) >= 3:
        b1 = buffer.get_bi(1)
        b2 = buffer.get_bi(2)
        b3 = buffer.get_bi(3)
        word_features = [b1.word_id, b2.word_id, b3.word_id]
        pos_features  = [b1.pos_id, b2.pos_id, b3.pos_id]

    elif len(buffer) == 2:
        b1 = buffer.get_bi(1)
        b2 = buffer.get_bi(2)
        word_features = [b1.word_id, b2.word_id, vocab.WORD_NULL]
        pos_features  = [b1.pos_id, b2.pos_id, vocab.POS_NULL]

    elif len(buffer) == 1:
        b1 = buffer.get_bi(1)
        word_features = [b1.word_id, vocab.WORD_NULL, vocab.WORD_NULL]
        pos_features  = [b1.pos_id, vocab.POS_NULL, vocab.POS_NULL]
    else:   #len(buffer) == 0
        word_features = [vocab.WORD_NULL, vocab.WORD_NULL, vocab.WORD_NULL]
        pos_features  = [vocab.POS_NULL, vocab.POS_NULL, vocab.POS_NULL]

    return word_features, pos_features
    


The remaining features have to do with the leftmost & rightmost children of the words at the top of the stack & buffer. Write the following 2 helper functions to make it easier to access these dependents:
* `get_lc(parser_config, word)`: Return a list of arcs to dependents of `word`, sorted from <b>left to right</b>. Only include dependents that are to the <b>left</b> of `word` in the sentence.
* `get_rc(parser_config, word)`: Return a list of arcs to dependents of `word`, sorted from <b>right to left</b>. Only include dependents that are to the <b>right</b> of `word` in the sentence.


In [ ]:
def get_lc(parser_config, word):
    '''
    Finds the left dependents of word, sorted from left to right.
    Returns:
        A list of Arcs whose head is word, sorted by the indices of the dependent word from left to right.
    '''

    arcs = []
    # use this "for loop" cooresponding to __iter__ defined in Dependencies class
    for arc in parser_config.dependencies: 
        if arc.head == word and arc.dependent.idx < word.idx: # dependent is left to head in the sentence
            if len(arcs) == 0:
                arcs.append(arc)
            else:   # len(arcs) >= 1
                for i in range(len(arcs)):
                    if arc.dependent.idx < arcs[i].dependent.idx:
                        arcs.insert(i, arc)
                        inserted = True
                        break
                if inserted == False:
                    arcs.append(arc)
    return arcs

In [ ]:
def get_rc(parser_config, word):
    '''
    Finds the right dependents of word, sorted from right to left.
    Returns:
        A list of Arcs whose head is word, sorted by the indices of the dependent word from right to left.
    '''

    arcs = []
    # use this "for loop" cooresponding to __iter__ defined in Dependencies class
    for arc in parser_config.dependencies:  
        if arc.head == word and arc.dependent.idx > word.idx: # dependent is right to head in the sentence
            if len(arcs) == 0:
                arcs.append(arc)
            else:   # len(arcs) >= 1
                for i in range(len(arcs)):
                    if arc.dependent.idx > arcs[i].dependent.idx:
                        arcs.insert(i, arc)
                        inserted = True
                        break
                if inserted == False:
                    arcs.append(arc)
    return arcs    


Let $lc_j(s_i)$ be the $j$th leftmost child of the $i$th item on the stack. Write the following function:
* `get_lc1_lc2_features(parser_config, i)`: Return word & POS features for $lc_1(s_i)$ and $lc_2(s_i)$. Additionally, return label features (the label ids) for the arcs that attach $lc_1(s_i)$ to $s_i$ and $lc_2(s_i)$ to $s_i$. As before, wherever a particular word does not exist, use the appropriate NULL token.

This function will be called with `i=1` and `i=2`, accounting for the words $lc_1(s_1)$, $lc_2(s_1)$, $lc_1(s_2)$, $lc_2(s_2)$.

In [ ]:
def get_lc1_lc2_features(parser_config, i):
    '''
    Get the word, POS, and label features for lc1(si) and lc2(si), where i in {1, 2}
    Returns:
        word_features: List of word ids for lc1(si), lc2(si) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for lc1(si), lc2(si) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for lc1(si), lc2(si) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}

    word = parser_config.stack.get_si(i)
    arcs = get_lc(parser_config, word)
    if len(arcs) == 0:
        word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2
    elif len(arcs) == 1:
        word_features = [arcs[0].dependent.word_id, parser_config.vocab.WORD_NULL]
        pos_features = [arcs[0].dependent.pos_id, parser_config.vocab.POS_NULL]
        label_features = [arcs[0].label_id, parser_config.vocab.LABEL_NULL]
    else:  #len(arcs) >= 2
        word_features = [arcs[0].dependent.word_id, arcs[1].dependent.word_id]
        pos_features = [arcs[0].dependent.pos_id, arcs[1].dependent.pos_id]
        label_features = [arcs[0].label_id, arcs[1].label_id]
        
    return word_features, pos_features, label_features


We will now write the analagous function for the rightmost children. Let $rc_j(s_i)$ be the $j$th rightmost child of the $i$th item on the stack. Write the following function:
* `get_rc1_rc2_features(parser_config, i)`: Return word & POS features for $rc_1(s_i)$ and $rc_2(s_i)$. Additionally, return label features (the label ids) for the arcs that attach $rc_1(s_i)$ to $s_i$ and $rc_2(s_i)$ to $s_i$. As before, wherever a particular word does not exist, use the appropriate NULL token.
This function will be called with `i=1` and `i=2`, accounting for the words $rc_1(s_1)$, $rc_2(s_1)$, $rc_1(s_2)$, $rc_2(s_2)$.

In [ ]:
def get_rc1_rc2_features(parser_config, i):
    '''
    Get the word, POS, and label features for rc1(si) and rc2(si), where i in {1, 2}
    Returns:
        word_features: List of word ids for rc1(si), rc2(si) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for rc1(si), rc2(si) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for rc1(si), rc2(si) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}

    word = parser_config.stack.get_si(i)
    arcs = get_rc(parser_config, word)
    if len(arcs) == 0:
        word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2
    elif len(arcs) == 1:
        word_features = [arcs[0].dependent.word_id, parser_config.vocab.WORD_NULL]
        pos_features = [arcs[0].dependent.pos_id, parser_config.vocab.POS_NULL]
        label_features = [arcs[0].label_id, parser_config.vocab.LABEL_NULL]
    else:  #len(arcs) >= 2
        word_features = [arcs[0].dependent.word_id, arcs[1].dependent.word_id]
        pos_features = [arcs[0].dependent.pos_id, arcs[1].dependent.pos_id]
        label_features = [arcs[0].label_id, arcs[1].label_id]

    return word_features, pos_features, label_features

Finally, write the following function:
* `get_llc_rrc_features(parser_config, i)`: Return word & POS features for $lc_1(lc_1(s_i))$ and $rc_1(rc_1(s_i))$. Additionally, return label features (the label ids) for the arcs that attach $lc_1(lc_1(s_i))$ to $lc_1(s_i)$ and $rc_1(rc_1(s_i))$ to $rc_1(s_i)$. As before, wherever a particular word does not exist, use the appropriate NULL token.

This function will be called with `i=1` and `i=2`, accounting for the words $lc_1(lc_1(s_1))$, $lc_1(lc_1(s_2))$, $rc_1(rc_1(s_1))$, $rc_1(rc_1(s_2))$.

In [ ]:
def get_llc_rrc_features(parser_config, i):
    '''
    Get the word, POS, and label features for lc1(lc1(si)), and rc1(rc1(si)), where i in {1, 2}
    Returns:
        word_features: List of word ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.WORD_NULL if a word does not exist)
        pos_features: List of POS ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.POS_NULL if a word does not exist)
        label_features: List of label ids for lc1(lc1(si)), and rc1(rc1(si)) (use vocab.LABEL_NULL if a word does not exist)
    '''
    assert i in {1,2}

    si = parser_config.stack.get_si(i)
    arcs = get_lc(parser_config, si)
    if len(arcs) == 0:
        l_wf = parser_config.vocab.WORD_NULL
        l_pf = parser_config.vocab.POS_NULL
        l_lf = parser_config.vocab.LABEL_NULL
    else:
        lc1_si = arcs[0].dependent
        arcs = get_lc(parser_config, lc1_si)
        if len(arcs) == 0:
            l_wf = parser_config.vocab.WORD_NULL
            l_pf = parser_config.vocab.POS_NULL
            l_lf = parser_config.vocab.LABEL_NULL
        else:
            # for lc1(lc1(si))
            l_wf = arcs[0].dependent.word_id
            l_pf = arcs[0].dependent.pos_id
            l_lf = arcs[0].label_id

    arcs = get_rc(parser_config, si)
    if len(arcs) == 0:
        r_wf = parser_config.vocab.WORD_NULL
        r_pf = parser_config.vocab.POS_NULL
        r_lf = parser_config.vocab.LABEL_NULL
    else:
        rc1_si = arcs[0].dependent
        arcs = get_rc(parser_config, rc1_si)
        if len(arcs) == 0:
            r_wf = parser_config.vocab.WORD_NULL
            r_pf = parser_config.vocab.POS_NULL
            r_lf = parser_config.vocab.LABEL_NULL
        else:
            # for rc1(rc1(si))
            r_wf = arcs[0].dependent.word_id
            r_pf = arcs[0].dependent.pos_id
            r_lf = arcs[0].label_id

    word_features = [l_wf, r_wf]
    pos_features = [l_pf, r_pf]
    label_features = [l_lf, r_lf]
    
    return word_features, pos_features, label_features


The following function `extract_features(parser_config)` calls each of these functions and returns a list of the 48 total features.

In [ ]:
def extract_features(parser_config): # for both train & inference

    word_features, pos_features, label_features = [], [], []

    # 1. Get word & pos features for s1, s2, and s3
    (x, y) = get_top3_stack_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y


    # 2. Get word & pos features for b1, b2, and b3
    (x, y) = get_top3_buffer_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y


    # 3. Get word & pos & label features for lc1(s1), lc1(s2), lc2(s1), lc2(s2)
    (x, y, z) = get_lc1_lc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_lc1_lc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    # 4. Get word & pos & label features for rc1(s1), rc1(s2), rc2(s1), rc2(s2)
    (x, y, z) = get_rc1_rc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_rc1_rc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    # 5. Get word & pos & label features for lc1(lc1(s1)), lc1(lc1(s2)), rc1(rc1(s1)), rc1(rc1(s2))
    (x, y, z) = get_llc_rrc_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    (x, y, z) = get_llc_rrc_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z


    features = word_features + pos_features + label_features
    assert len(features) == 48
    return features

# Step 3: Dataset & Model

Define the Pytorch `Dataset` class as well as the model.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Instantiate Dataset

Create a Pytorch `Dataset` to be used to feed  training data to the model.

In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, data, vocab):
        self.X = np.array([d[0] for d in data])
        self.y = np.array([vocab.tran2id[d[1]] for d in data])
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

## Define Model

Here we implement the `__init(...)__` and `forward(...)` methods of a feed-forward network. The network will have an embedding layer, a single hidden layer, and an output layer. The `forward(...)` method will take in the features we have extracted from the parser configuration and predict the next parser action.

In [ ]:
class Model(nn.Module):
    def __init__(self, num_embeddings, embed_size, n_features, hidden_size, n_classes, dropout_prob):
        '''
        Initialize the weights of feed-forward neural network.
        Args:
            num_embeddings: Number of embedding vectors in embedding layer (int)
            embed_size: Size of the embedding vectors in embedding layer (int)
            n_features: Number of features in the input to the model (int)
            hidden_size: Hidden size (int)
            n_classes: Number of classes in output (int)
            dropout_prob: Probability of dropout (float)
        '''
        super(Model, self).__init__()


        # Initialize embedding layer 
        # read documentation for nn.Embedding for thee parameters
        self.EmbeddingLayer = nn.Embedding(num_embeddings, embed_size)

        # Initialize a linear layer that maps the (concatenated) embeddings to a single vector of size hidden_size
        # figure out the parameters: read nn.Linear*(), also refer to step (2) in forward() function
        # below, the parameter in_features should be n_features * embed_size;
        # The parameter out_features should be "a single vector of size hidden_size" as mentioned above
        self.LinearLayer1 = nn.Linear(n_features * embed_size, hidden_size)
       
        # Create a dropout layer with dropout_prob
        self.DropoutLayer = nn.Dropout(dropout_prob)

        # Initialize a linear layer that maps the hidden vector to the number of output classes
        self.LinearLayer2 = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        '''
        This function predicts the next parser action, given the features extracted from the current parser state.
        Inputs:
             x: input features, [batch_size, n_features]
        Returns:
            logits: [batch_size, n_classes]
        '''

   
        # (1) Obtain embedding vectors for the input
        #            - Output size: [batch_size, n_features, embed_size]
        # reading nn.Embedding documention reveals that the output's first 
        # two dimensions should be the same as x - [batch_size, n_features], and the 3rd dimension 
        # should be embed_size, so the output should be [batch_size, n_features, embed_size] 
        
        embedding = self.EmbeddingLayer(x)  #[batch_size, n_features, embed_size]
        # added comment - in the following we first permute/swap the 2nd and 3rd dimension 
        embedding = embedding.permute(0, 2, 1) # [batch_size, embed_size, n_features]
        # then use view(batch_size, -1) to concat all n_feature embeddings, each of size embed_size
        # the permute performed above is necessary, otherwise it will concat along the n_feature dimension 
        # in the following the -1 means concat the last two dimentions to one, google and see details 
        # the call of contiguous() is also necessary 
        batch_size = embedding.size(dim = 0)
        embedding = embedding.contiguous().view(batch_size, -1) #[batch_size, embed_size * n_features]

        # (2) Pass the result through the first linear layer and apply ReLU activation
        LL1_output = self.LinearLayer1(embedding)  #[batch_size, hidden_size]
        LL1_ReLU_output = F.relu(LL1_output)
        
        # (3) Apply dropout
        dropout_output = self.DropoutLayer(LL1_ReLU_output)

        #(4) Pass the result through the final linear layer and return its output (do NOT call softmax!)
        #            - Output size: [batch_size, n_classes]
        output = self.LinearLayer2(dropout_output)

        return output

# Step 4: Train Model


In [ ]:
import math
from torch import optim
from tqdm.notebook import tqdm
import time

First, read in the training dataset, create training examples, extract features, and instantiate the Pytorch `Dataset`.

In [ ]:
def prepare_data(train_name='train', test_name='test'):

    train_set, test_set = load_data()

    vocab = Vocabulary(train_set)
    vocab.printStats()
    print()

    train_set = vocab.buildSentences(train_set)
    test_set = vocab.buildSentences(test_set)

    train_examples = generate_all_training_examples(vocab, train_set, feat_extract=extract_features)

    return vocab, train_examples, test_set

In [ ]:
if __name__== "__main__":
    vocab, train_examples, test_data = prepare_data()
    train_dataset = TrainDataset(train_examples, vocab)

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


The neural network is trained with cross-entropy loss.

In [ ]:
def train_model(model, vocab, train_data_loader, optimizer, n_epochs, device):
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):
        start = time.time()
        n_batch = 0
        total_loss = 0
        model.train()      
        for train_x, train_y in tqdm(train_data_loader):
            optimizer.zero_grad() 
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            logits = model(train_x)
            loss = loss_func(logits, train_y)
            loss.backward()
            optimizer.step()
            
            total_loss +=  loss.item()
            n_batch += 1
        
        print('Epoch:{:2d}/{}\t Loss: {:.4f} \t({:.2f}s)'.format(epoch + 1, n_epochs, total_loss / n_batch, time.time() - start))

Next instantiate the model and an <a href=https://jmlr.org/papers/volume12/duchi11a/duchi11a.pdf>Adagrad</a> optimizer

In [ ]:
def count_parameters(model):
    """
    Count number of trainable parameters in the model
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
if __name__ == "__main__":
    # HYPERPARAMETERS
    BATCH_SIZE = 1024
    LEARNING_RATE = 0.01
    N_EPOCHS = 10
    HIDDEN_SIZE = 300
    DROPOUT_PROB = 0.1
    EMBED_SIZE = 100
    WEIGHT_DECAY = 1e-8
    N_EMBEDDINGS = vocab.n_tokens # Do not change!
    N_FEATURES = 48 # Do not change!
    N_CLASSES = vocab.n_trans # Do not change!
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
    model = Model(N_EMBEDDINGS, EMBED_SIZE, N_FEATURES, HIDDEN_SIZE, N_CLASSES, DROPOUT_PROB).to(device)
    optimizer = optim.Adagrad(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    print('The model has {:,d} trainable parameters'.format(count_parameters(model)))


The model has 3,142,395 trainable parameters


Run the cell below to train the model.

In [ ]:
if __name__=='__main__':
    train_model(model, vocab, train_data_loader, optimizer, N_EPOCHS, device)

  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 1/10	 Loss: 0.7875 	(75.47s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 2/10	 Loss: 0.2572 	(73.42s)


  0%|          | 0/364 [00:00<?, ?it/s]

...

Epoch: 8/10	 Loss: 0.1483 	(73.43s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 9/10	 Loss: 0.1412 	(73.60s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch:10/10	 Loss: 0.1354 	(73.87s)


# Step 5: Evaluate Model

Now the model has been trained, it can be used to parse unseen sentences from a test set.

## Select Best Legal Prediction
It is possible that the model will predict an illegal action. For example, the model may predict `S` (shift) when the buffer is empty, which is not a valid move. There is no guarantee that this will not happen for an unseen sentence, and so that possibility has to be accounted at inference time.

Thus,the <b>legal</b> action with the highest probability for each parser configuration should be returned. 

In [ ]:
def select_best_legal_action(parser_configs, predictions, n_labels):
    '''
    Returns the highest probability **legal** prediction for each parser configuration.
    Inputs:
        parser_configs: list of parser configurations of length N
        predictions: np.array of size [N, 2*n_labels + 1]
        n_labels: int, the number of labels in our model
    Returns:
        preds: np.array of length N, containing the indices of the highest probability legal action for each example
    '''

    assert len(parser_configs) == len(predictions)
    for i in range(len(parser_configs)):
        if len(parser_configs[i].buffer) == 0:   
            # buffer is empty, cannot do shift, so set the prbability for shift to 0
            predictions[i][2*n_labels] = 0.0

        if len(parser_configs[i].stack) == 1:   
            # stack has only one item, ROOT, cannot perform LeftArc or RightArc, so set their probabilitis to 0
            predictions[i][:2*n_labels] = [0.0]*2*n_labels
        elif len(parser_configs[i].stack) == 2:
            # stack has exactly two items, that will be ROOT and another words to its right
            # cannot perform LeftArc in this case, so set those probabilitis to 0
            predictions[i][:n_labels] = [0.0]*n_labels
        # It will never happen that predictions[i] is all 0, since the parser_config with empty buffer 
        # and a stack with only ROOT will not be passed to this function, this is "DONE", inference will halt when that happens      
    preds = np.argmax(predictions, axis = 1) 

    return preds

The following function takes a (trained) model and makes the best legal prediction for a batch of parser configurations.

In [ ]:
def predict(model, vocab, parser_configs):
    '''
    Predicts the next transition for each ParserConfiguration in the batch (`parsers`).
    '''
    model_device = next(model.parameters()).device
    
    x = np.array([extract_features(p) for p in parser_configs])
    x = torch.from_numpy(x).long().to(model_device)
    
    with torch.no_grad():
        pred = model(x)

    pred = pred.detach().cpu().numpy()
    pred = select_best_legal_action(parser_configs, pred, vocab.n_labels)
    actions = [vocab.id2tran[p] for p in pred]
    return actions

## Test Set Attachment Score

The following functions use the model to parse all sentences in the test set, and compute the attachment score. The <b>unlabeled attachment score</b> is the percentage of arcs in the test set for which the model gets the head correct. The <b>labeled attachment score</b> is the percentage of arcs for which the model gets <em>both</em> the head and the label correct. Thus, attachment score is a number between 0 and 100, and a higher score is better.

In [ ]:
def run_inference(sentences, model, vocab, batch_size=2000):
    '''
    Infers the dependency parse for each sentence given a trained model.
    '''
    N = len(sentences)

    # Initialize parser configs
    parser_configs = [None] * N
    for i in range(N):
        sent = sentences[i]
        parser_config = ParserConfiguration(sent, vocab)
        parser_configs[i] = parser_config

    parses_completed = [False] * N # Indicates whether a given parse is completed
    
    while sum(parses_completed) != N:

        # Get batch along with indices
        batch_idxes = []
        for idx in range(N):
            if not parses_completed[idx]: batch_idxes.append(idx)
            if len(batch_idxes) == batch_size: break
        batch = [parser_configs[idx] for idx in batch_idxes]

        # Make prediction, run a parse step, and check for completion
        transitions = predict(model, vocab, batch)
        for idx, parser, transition in zip(batch_idxes, batch, transitions):
            parser.parse_step(transition)
            if not parser.buffer.buffer and len(parser.stack.stack) == 1:
                parses_completed[idx] = True
    
    return [parser.dependencies for parser in parser_configs]

def transform_to_head_label(dependencies):
    head = [-1] * len(dependencies.arcs)
    label = [-1] * len(dependencies.arcs)
    for dep in dependencies.arcs:
        head[dep.dependent.idx-1] = dep.head.idx
        label[dep.dependent.idx-1] = dep.label_id  
    return head, label


def evaluate(model, vocab, dataset, eval_batch_size=5000):
    model.eval()
    sentences = [x[0] for x in dataset]
    gold_dependencies = [x[1] for x in dataset]
    pred_dependencies = run_inference(sentences, model, vocab, eval_batch_size)
    
    UAS, LAS = 0.0, 0.0

    all_tokens = 0
    
    for i in range(len(gold_dependencies)):
        assert len(gold_dependencies[i].arcs) == len(pred_dependencies[i].arcs)
        
        # Get gold answers
        gold_head, gold_label = transform_to_head_label(gold_dependencies[i])
        
        # Get predictions
        pred_head, pred_label = transform_to_head_label(pred_dependencies[i])
        
        assert len(gold_head) == len(pred_head) and len(gold_label) == len(pred_label)
        assert -1 not in gold_head + gold_label + pred_head + pred_label

        for pred_h, gold_h, pred_l, gold_l  in zip(pred_head, gold_head, pred_label, gold_label):
            UAS += (1 if pred_h == gold_h else 0)
            LAS += (1 if pred_h == gold_h and pred_l == gold_l else 0)
            all_tokens += 1
    return UAS / all_tokens * 100, LAS / all_tokens * 100, pred_dependencies

Run the following cell to calculate the attachment scores. 

In [ ]:
if __name__=="__main__":
    UAS, LAS, test_predictions = evaluate(model, vocab, test_data)
    print("Test Set Unlabeled Attachment Score:", UAS)
    print("Test Set Labeled Attachment Score:", LAS)

Test Set Unlabeled Attachment Score: 83.19945920152696
Test Set Labeled Attachment Score: 80.67043104819469
